In [1]:
# Packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score
import statsmodels.api as sm
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import libpysal
import spreg
import esda
import geopandas as gpd
from pysal.model import mgwr

In [2]:
df = pd.read_csv("../Data/shp/modeling.csv")
gdf = gpd.read_file("../Data/shp/merge_all.shp")

In [3]:
df['Geography'] = df['Geography'].astype(str)
merge = pd.merge(gdf, df, left_on="geoid20", right_on="Geography", how="right")

len(merge)

286

In [4]:
merge.head()

,geoid20,Area,GEOID20_1,A311,A311_his,property,A311_p,A311_his_p,X,Y,...,percent_singleunits,percent_multiunit,percent_mobile_homes,percent_owneroccupiedunit,percent_crowding,percent_group_quarters,median_year_properties_built,median_value_properties_built,percent_no_vehicle,311_index_per_property
0,360290001101,0.672269,360290001101,23,57,215,10.697674,26.511628,1.082646e+06,1.033333e+06,...,0.503125,0.0,0.00000,0.804054,0.000000,0.007463,1926.0,84000.0,0.000000,10.697674
1,360290001102,0.381606,360290001102,29,102,449,6.458797,22.717149,1.082659e+06,1.035794e+06,...,0.797546,0.0,0.00000,0.702765,0.069124,0.000000,1930.0,84000.0,0.225806,6.458797
2,360290001103,8.632217,360290001103,27,113,244,11.065574,46.311475,1.077492e+06,1.036705e+06,...,0.664251,0.0,0.00000,0.540193,0.000000,0.000000,1910.0,63000.0,0.070740,11.065574
3,360290002001,0.422595,360290002001,21,116,270,7.777778,42.962963,1.082870e+06,1.041362e+06,...,0.146789,0.0,0.00000,0.356201,0.042216,0.005164,1920.0,76000.0,0.155673,7.777778
4,360290002002,0.197490,360290002002,35,82,292,11.986301,28.082192,1.082900e+06,1.038932e+06,...,0.320755,0.0,0.02965,0.314935,0.000000,0.000000,1910.0,77000.0,0.324675,11.986301


## GWR 

In [21]:
X_gwr = merge
X_gwr = X_gwr.set_index("Geography")
y = X_gwr.pop('311_index_per_property')
X = X_gwr[[
         'his_num_311_per_property',
         'neighbor_his_num_per_property',
         'snow_depth',
         'percent_below_poverty',
         'percent_civilian_unemployed',
         'per_capita_income',
         'percent_no_highschool',
         'percent_65older',
         'percent_17younger',
         'percent_household_disability',
         'percent_single_parent_household',
         'percent_minority',
         'percent_notwell_english',
         'percent_singleunits',
         'percent_multiunit',
         'percent_mobile_homes',
         'percent_owneroccupiedunit',
         'percent_crowding',
         'percent_group_quarters',
         'median_year_properties_built',
         'median_value_properties_built',
         'percent_no_vehicle'
        ]]

#lng_lat_coords = np.column_stack([pd.to_numeric(X_gwr["X"]), pd.to_numeric(X_gwr["Y"])])
X_gwr["X"] = pd.to_numeric(X_gwr['X']) 
X_gwr["Y"] = pd.to_numeric(X_gwr['Y']) 
lng_lat_coords = X_gwr[["X","Y"]]

In [22]:
lng_lat_coords

,X,Y
Geography,,
360290001101,1.082646e+06,1.033333e+06
360290001102,1.082659e+06,1.035794e+06
360290001103,1.077492e+06,1.036705e+06
360290002001,1.082870e+06,1.041362e+06
360290002002,1.082900e+06,1.038932e+06
...,...,...
360290171001,1.062837e+06,1.066882e+06
360290171002,1.066035e+06,1.066475e+06
360290171003,1.066214e+06,1.065489e+06


In [23]:
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW

In [24]:
y_gwr_predict = []
y_true = []

ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)

fold_index = 0

avg_r_squared = 0

gwr_bw = 257 # the optimal bw is 285 in the previous code; however, the package doesn't allow us to set a bandwidth higher than 257 in ten-fold cross validation

for train_index, test_index in ten_fold.split(X_gwr):
    print("TEST:", test_index)
    fold_index += 1

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    coord_train, coord_test = lng_lat_coords.iloc[train_index], lng_lat_coords.iloc[test_index]
    coord_train_np = np.array(list(zip(coord_train["X"], coord_train["Y"])))
    coord_test_np = np.array(list(zip(coord_test["X"], coord_test["Y"])))
    #print(coord_test_np)
    
    training_stat = X_train.describe().transpose()
    scaled_X_train = standarize_data(X_train, training_stat)
    scaled_X_test = standarize_data(X_test, training_stat)

    #gwr_selector = Sel_BW(coord_train_np, np.asarray(y_train).reshape(-1,1), np.asarray(scaled_X_train))
    #gwr_bw = gwr_selector.search(bw_min=2)
    gwr_model = GWR(coord_train_np, np.asarray(y_train).reshape(-1,1), np.asarray(scaled_X_train), gwr_bw)
    gwr_results = gwr_model.fit()
    y_train_pred = gwr_results.predy.flatten()
    r2_train = r2_score(y_train, y_train_pred)
    print("training r2: "+str(r2_train))
    
    # use gwr to make predictions
    scale = gwr_results.scale
    residuals = gwr_results.resid_response
    pred_results = gwr_model.predict(coord_test_np, np.asarray(scaled_X_test), scale, residuals)
    this_y_predict = pred_results.predictions.flatten()

    y_gwr_predict = y_gwr_predict + this_y_predict.tolist()
    y_true = y_true + y_test.tolist()
    print(r2_score(y_test, this_y_predict))
    avg_r_squared += r2_score(y_test, this_y_predict)

TEST: [  5   9  33  45  56  73  79 109 111 124 143 146 155 170 185 196 200 212
 217 220 227 233 251 265 267 268 274 275 283]
training r2: 0.6004323364596478
0.3741024356795898
TEST: [  6  22  24  30  42  46  60  63  75  77  84  92  93 116 147 164 175 177
 181 193 203 204 207 211 221 234 240 249 269]
training r2: 0.5938289207832568
0.004708694914430378
TEST: [ 10  15  16  18  19  25  37  66  67  68  82  86  90 112 113 117 120 125
 139 144 154 159 165 202 219 246 250 255 258]
training r2: 0.5944751552561811
0.02144528481802288
TEST: [  2  31  38  55  57  69  78  97 101 104 108 114 119 126 132 148 152 167
 173 184 206 218 224 232 254 264 272 277 281]
training r2: 0.5976138487183256
0.02462175741100625
TEST: [ 12  29  35  65  74  76  85  96 107 115 118 127 137 140 157 158 168 172
 176 179 183 192 194 216 223 238 239 278 284]
training r2: 0.5998124490913807
0.000246358361929766
TEST: [  0  11  26  28  36  41  51  61  95  98 100 136 141 142 150 178 180 186
 195 210 215 225 226 231 242 256 26

In [25]:
gwr_rmse = root_mean_squared_error(y_true , y_gwr_predict)
gwr_r2 = r2_score(y_true, y_gwr_predict)

print("RMSE: "+str(gwr_rmse))
print("R2: "+str(gwr_r2))

RMSE: 4.236582546890085
R2: 0.3660545619224028
